This The implementation of the S-DES algorithm:


1. P-BOX:


In [48]:
initial_p_box_mapping = [2,6,3,1,4,8,5,7]
final_p_box_mapping = [4,1,3,5,7,2,8,6]
compression_p_box_mapping = [6,3,7,4,8,5,10,9]
straight_p_box_mapping = [3,5,2,7,4,10,1,9,8,6]
straight_p_box_mapping_small = [2,4,3,1]
expansion_p_box_mapping = [4,1,2,3,2,3,4,1]
def generalBox(data,mapping):
    return [data[i - 1] for i in mapping]

2. Key generation:


In [49]:
def leftShift(data,numberOfShifts):
    for shifts in range(numberOfShifts):
        data.append(data.pop(0))
    return data

def keyGeneration(key):
    keys = []
    data = [*str(key)]
    data = generalBox(data,straight_p_box_mapping)
    
    k1 = data[:5]
    k2 = data[5:]
    
    k1 = leftShift(k1, 1)
    k2 = leftShift(k2, 1)
    
    keys.append(generalBox(k1+k2,compression_p_box_mapping))
    
    k1 = leftShift(k1, 2)
    k2 = leftShift(k2, 2)
    
    keys.append(generalBox(k1+k2,compression_p_box_mapping))
    
    return keys

3. The S-DES round:


In [50]:
one_s_box_mapping = [["01", "00", "11", "10"], ["11", "10", "01", "00"], [
    "00", "10", "01", "11"], ["11", "01", "11", "10"]]
two_s_box_mapping = [["00", "01", "10", "11"], ["10", "00", "01", "11"], [
    "11", "00", "01", "00"], ["10", "01", "00", "11"]]


def sBox_permutation(data, mapping):
    i = int('00'+(data[0]) + (data[3]), 2)
    j = int('00'+(data[1]) + (data[2]), 2)
    return mapping[i][j]


def function(r, key):

    r = generalBox(r, expansion_p_box_mapping)

    xor_result = [str((ord(a) ^ ord(b))) for a, b in zip(r, key)]
    
    k1 = xor_result[:4]
    k2 = xor_result[4:]
    
    sBoxResult = sBox_permutation(
        k1, one_s_box_mapping) + sBox_permutation(k2, two_s_box_mapping)
        
    return generalBox([*str(sBoxResult)], straight_p_box_mapping_small)


def mixer(l, r, key):

    function_result = function(r, key)
    
    xor_result = [str((ord(a) ^ ord(b))) for a, b in zip(l, function_result)]

    return [xor_result, r]


def Round(l, r, key, withSwapper):
    mixerResult = mixer(l, r, key)
    if(withSwapper):
        return [mixerResult[1], mixerResult[0]]
    else:
        return mixerResult


4. Compine everything to encrpyt and decrypt:

In [51]:
def encrypt_sdes(data,key):
    initial_permutation = generalBox(data,initial_p_box_mapping)
    keys = keyGeneration(key)
    round_result_one = Round(initial_permutation[:4],initial_permutation[4:],keys[0],True)
    round_result_two = Round(round_result_one[0],round_result_one[1],keys[1],False)
    return generalBox(round_result_two[0]+round_result_two[1],final_p_box_mapping)

def decrypt_sdes(data,key):
    initial_permutation = generalBox(data,initial_p_box_mapping)
    keys = keyGeneration(key)
    round_result_one = Round(initial_permutation[:4],initial_permutation[4:],keys[1],True)
    round_result_two = Round(round_result_one[0],round_result_one[1],keys[0],False)
    return generalBox(round_result_two[0]+round_result_two[1],final_p_box_mapping)

Testing S-Des:

In [52]:
import time
files = ['25K','50K','1MB','250MB','500MB'
]
key = '0111100011'
for file in files:
    print('file: {}'.format(file))
    f = open(file.format(file), "r")
    fileData = f.read()
    e = open("encryption_{}".format(file), "a")
    read_e = open("encryption_{}".format(file), "r")
    d = open("decryption_{}".format(file), "a")
    loop = 0
    start = time.time()
    for data in fileData:
        loop = loop + 1
        if loop % 8 == 0:
            e.write(''.join(encrypt_sdes(fileData[loop - 8: loop],key)))
    end = time.time()
    print('time to encrypt: {}'.format(end-start))

    fileData = read_e.read()
    loop = 0
    start = time.time()
    for data in fileData:
        loop = loop + 1
        if loop % 8 == 0:
            d.write(''.join(decrypt_sdes(fileData[loop - 8: loop],key)))
    end = time.time()
    print('time to decrypt: {}'.format(end-start))

file: 25K
time to encrypt: 0.0953218936920166
time to decrypt: 0.08541488647460938
file: 50K
time to encrypt: 0.17817401885986328
time to decrypt: 0.17455387115478516
file: 1MB
time to encrypt: 3.804630994796753
time to decrypt: 4.006036043167114
file: 250MB
time to encrypt: 882.1362218856812
time to decrypt: 874.1538519859314
file: 500MB
time to encrypt: 1734.093581199646
time to decrypt: 1734.4765238761902


Des implementation:

In [53]:
# Hexadecimal to binary conversion
def hex2bin(s):
	mp = {'0' : "0000",
		'1' : "0001",
		'2' : "0010",
		'3' : "0011",
		'4' : "0100",
		'5' : "0101",
		'6' : "0110",
		'7' : "0111",
		'8' : "1000",
		'9' : "1001",
		'A' : "1010",
		'B' : "1011",
		'C' : "1100",
		'D' : "1101",
		'E' : "1110",
		'F' : "1111" }
	bin = ""
	for i in range(len(s)):
		bin = bin + mp[s[i]]
	return bin
	
# Binary to hexadecimal conversion
def bin2hex(s):
	mp = {"0000" : '0',
		"0001" : '1',
		"0010" : '2',
		"0011" : '3',
		"0100" : '4',
		"0101" : '5',
		"0110" : '6',
		"0111" : '7',
		"1000" : '8',
		"1001" : '9',
		"1010" : 'A',
		"1011" : 'B',
		"1100" : 'C',
		"1101" : 'D',
		"1110" : 'E',
		"1111" : 'F' }
	hex = ""
	for i in range(0,len(s),4):
		ch = ""
		ch = ch + s[i]
		ch = ch + s[i + 1]
		ch = ch + s[i + 2]
		ch = ch + s[i + 3]
		hex = hex + mp[ch]
		
	return hex

# Binary to decimal conversion
def bin2dec(binary):
	
	binary1 = binary
	decimal, i, n = 0, 0, 0
	while(binary != 0):
		dec = binary % 10
		decimal = decimal + dec * pow(2, i)
		binary = binary//10
		i += 1
	return decimal

# Decimal to binary conversion
def dec2bin(num):
	res = bin(num).replace("0b", "")
	if(len(res)%4 != 0):
		div = len(res) / 4
		div = int(div)
		counter =(4 * (div + 1)) - len(res)
		for i in range(0, counter):
			res = '0' + res
	return res

# Permute function to rearrange the bits
def permute(k, arr, n):
	permutation = ""
	for i in range(0, n):
		permutation = permutation + k[arr[i] - 1]
	return permutation

# shifting the bits towards left by nth shifts
def shift_left(k, nth_shifts):
	s = ""
	for i in range(nth_shifts):
		for j in range(1,len(k)):
			s = s + k[j]
		s = s + k[0]
		k = s
		s = ""
	return k

# calculating xow of two strings of binary number a and b
def xor(a, b):
	ans = ""
	for i in range(len(a)):
		if a[i] == b[i]:
			ans = ans + "0"
		else:
			ans = ans + "1"
	return ans

# Table of Position of 64 bits at initial level: Initial Permutation Table
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
				60, 52, 44, 36, 28, 20, 12, 4,
				62, 54, 46, 38, 30, 22, 14, 6,
				64, 56, 48, 40, 32, 24, 16, 8,
				57, 49, 41, 33, 25, 17, 9, 1,
				59, 51, 43, 35, 27, 19, 11, 3,
				61, 53, 45, 37, 29, 21, 13, 5,
				63, 55, 47, 39, 31, 23, 15, 7]

# Expansion D-box Table
exp_d = [32, 1 , 2 , 3 , 4 , 5 , 4 , 5,
		6 , 7 , 8 , 9 , 8 , 9 , 10, 11,
		12, 13, 12, 13, 14, 15, 16, 17,
		16, 17, 18, 19, 20, 21, 20, 21,
		22, 23, 24, 25, 24, 25, 26, 27,
		28, 29, 28, 29, 30, 31, 32, 1 ]

# Straight Permutation Table
per = [ 16, 7, 20, 21,
		29, 12, 28, 17,
		1, 15, 23, 26,
		5, 18, 31, 10,
		2, 8, 24, 14,
		32, 27, 3, 9,
		19, 13, 30, 6,
		22, 11, 4, 25 ]

# S-box Table
sbox = [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
		[ 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
		[ 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
		[15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13 ]],
			
		[[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
			[3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
			[0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
		[13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9 ]],

		[ [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
		[13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
		[13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
			[1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12 ]],
	
		[ [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
		[13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
		[10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
			[3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14] ],
		
		[ [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
		[14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
			[4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
		[11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3 ]],
	
		[ [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
		[10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
			[9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
			[4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13] ],
		
		[ [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
		[13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
			[1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
			[6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12] ],
		
		[ [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
			[1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
			[7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
			[2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11] ] ]

# Final Permutation Table
final_perm = [ 40, 8, 48, 16, 56, 24, 64, 32,
			39, 7, 47, 15, 55, 23, 63, 31,
			38, 6, 46, 14, 54, 22, 62, 30,
			37, 5, 45, 13, 53, 21, 61, 29,
			36, 4, 44, 12, 52, 20, 60, 28,
			35, 3, 43, 11, 51, 19, 59, 27,
			34, 2, 42, 10, 50, 18, 58, 26,
			33, 1, 41, 9, 49, 17, 57, 25 ]

def encrypt_des(pt, rkb, rk):
	pt = hex2bin(pt)
	
	# Initial Permutation
	pt = permute(pt, initial_perm, 64)
	
	# Splitting
	left = pt[0:32]
	right = pt[32:64]
	for i in range(0, 16):
		# Expansion D-box: Expanding the 32 bits data into 48 bits
		right_expanded = permute(right, exp_d, 48)
		
		# XOR RoundKey[i] and right_expanded
		xor_x = xor(right_expanded, rkb[i])

		# S-boxex: substituting the value from s-box table by calculating row and column
		sbox_str = ""
		for j in range(0, 8):
			row = bin2dec(int(xor_x[j * 6] + xor_x[j * 6 + 5]))
			col = bin2dec(int(xor_x[j * 6 + 1] + xor_x[j * 6 + 2] + xor_x[j * 6 + 3] + xor_x[j * 6 + 4]))
			val = sbox[j][row][col]
			sbox_str = sbox_str + dec2bin(val)
			
		# Straight D-box: After substituting rearranging the bits
		sbox_str = permute(sbox_str, per, 32)
		
		# XOR left and sbox_str
		result = xor(left, sbox_str)
		left = result
		
		# Swapper
		if(i != 15):
			left, right = right, left
	
	# Combination
	combine = left + right
	
	# Final permutation: final rearranging of bits to get cipher text
	cipher_text = permute(combine, final_perm, 64)
	return cipher_text

Testing Des:

In [57]:
import time
files = ['25k','50k','1mb','250mb','500mb']
for file in files:
	print('file: {}'.format(file))
	f = open(file.format(file), "r")
	e = open("encryption_{}_des".format(file), "a")
	read_e = open("encryption_{}_des".format(file), "r")
	d = open("decryption_{}_des".format(file), "a")
	pt = f.read()
	key = "AABB09182736CCDD"

	# Key generation
	# --hex to binary
	key = hex2bin(key)

	# --parity bit drop table
	keyp = [57, 49, 41, 33, 25, 17, 9,
	1, 58, 50, 42, 34, 26, 18,
	10, 2, 59, 51, 43, 35, 27,
	19, 11, 3, 60, 52, 44, 36,
	63, 55, 47, 39, 31, 23, 15,
	7, 62, 54, 46, 38, 30, 22,
	14, 6, 61, 53, 45, 37, 29,
	21, 13, 5, 28, 20, 12, 4 ]

	# getting 56 bit key from 64 bit using the parity bits
	key = permute(key, keyp, 56)

	# Number of bit shifts
	shift_table = [1, 1, 2, 2,
			2, 2, 2, 2,
			1, 2, 2, 2,
			2, 2, 2, 1 ]

	# Key- Compression Table : Compression of key from 56 bits to 48 bits
	key_comp = [14, 17, 11, 24, 1, 5,
		3, 28, 15, 6, 21, 10,
		23, 19, 12, 4, 26, 8,
		16, 7, 27, 20, 13, 2,
		41, 52, 31, 37, 47, 55,
		30, 40, 51, 45, 33, 48,
		44, 49, 39, 56, 34, 53,
		46, 42, 50, 36, 29, 32 ]

	# Splitting
	left = key[0:28] # rkb for RoundKeys in binary
	right = key[28:56] # rk for RoundKeys in hexadecimal

	rkb = []
	rk = []
	for i in range(0, 16):
	# Shifting the bits by nth shifts by checking from shift table
		left = shift_left(left, shift_table[i])
		right = shift_left(right, shift_table[i])

	# Combination of left and right string
		combine_str = left + right

	# Compression of key from 56 to 48 bits
		round_key = permute(combine_str, key_comp, 48)

		rkb.append(round_key)
		rk.append(bin2hex(round_key))
	
	print("Encryption")
	start = time.time()
	cipher_text = bin2hex(encrypt_des(pt, rkb, rk))
	end = time.time()
	e.write(cipher_text)
	print('time to encrypt: {}'.format(end-start))

	print("Decryption")
	rkb_rev = rkb[::-1]
	rk_rev = rk[::-1]
	pt = read_e.read()
	start = time.time()
	text = bin2hex(encrypt_des(cipher_text, rkb_rev, rk_rev))
	end = time.time()
	d.write(text)
	print('time to decryption: {}'.format(end-start))

file: 25k
Encryption
time to encrypt: 0.0050029754638671875
Decryption
time to decryption: 0.0007488727569580078
file: 50k
Encryption
time to encrypt: 0.008219003677368164
Decryption
time to decryption: 0.0008382797241210938
file: 1mb
Encryption
time to encrypt: 0.15034198760986328
Decryption
time to decryption: 0.0007832050323486328
file: 250mb
Encryption
time to encrypt: 43.44892621040344
Decryption
time to decryption: 0.0008270740509033203
file: 500mb
Encryption
time to encrypt: 86.90203595161438
Decryption
time to decryption: 0.0008099079132080078


RSA key generate:

In [58]:
import random, sys, os
import random

def gcd(a, b):
    # Return the Greatest Common Divisor of a and b using Euclid's Algorithm
    while a != 0:
        a, b = b % a, a
    return b


def findModInverse(a, m):
    # Return the modular inverse of a % m, which is
    # the number x such that a*x % m = 1

    if gcd(a, m) != 1:
        return None # No mod inverse exists if a & m aren't relatively prime.

    # Calculate using the Extended Euclidean Algorithm:
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # Note that // is the integer division operator
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

def rabinMiller(num):
    # Returns True if num is a prime number.

    s = num - 1
    t = 0
    while s % 2 == 0:
        # Keep halving s until it is odd (and use t
        # to count how many times we halve s)
        s = s // 2
        t += 1

    for trials in range(5): # Try to falsify num's primality 5 times.
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # (This test does not apply if v is 1.)
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True


def isPrime(num):
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabinMiller().

    if (num < 2):
        return False # 0, 1, and negative numbers are not prime

    # About 1/3 of the time we can quickly determine if num is not prime
    # by dividing by the first few dozen prime numbers. This is quicker
    # than rabinMiller(), but unlike rabinMiller() is not guaranteed to
    # prove that a number is prime.
    lowPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]

    if num in lowPrimes:
        return True

    # See if any of the low prime numbers can divide num:
    for prime in lowPrimes:
        if (num % prime == 0):
            return False

    # If all else fails, call rabinMiller() to determine if num is a prime:
    return rabinMiller(num)


def generateLargePrime(keysize=1024):
    # Return a random prime number that is keysize bits in size.
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        if isPrime(num):
            return num
def main():
    # Create a public/private keypair with 1024 bit keys:
    makeKeyFiles('test_key', 1024)

def generateKey(keySize):
    # Creates a public/private keys keySize bits in size.
    p = 0
    q = 0
    # Step 1: Create two prime numbers, p and q. Calculate n = p * q.
    print('Generating p & q primes...')
    while p == q:
        p = generateLargePrime(keySize)
        q = generateLargePrime(keySize)
    n = p * q

    # Step 2: Create a number e that is relatively prime to (p-1)*(q-1):
    print('Generating e that is relatively prime to (p-1)*(q-1)...')
    while True:
        # Keep trying random numbers for e until one is valid:
        e = random.randrange(2 ** (keySize - 1), 2 ** (keySize))
        if gcd(e, (p - 1) * (q - 1)) == 1:
            break

    # Step 3: Calculate d, the mod inverse of e:
    print('Calculating d that is mod inverse of e...')
    d = findModInverse(e, (p - 1) * (q - 1))

    publicKey = (n, e)
    privateKey = (n, d)

    print('Public key:', publicKey)
    print('Private key:', privateKey)

    return (publicKey, privateKey)


def makeKeyFiles(name, keySize):
    if os.path.exists('%s_pubkey.txt' % (name)) or os.path.exists('%s_privkey.txt' % (name)):
        sys.exit('WARNING: The file %s_pubkey.txt or %s_privkey.txt already exists! Use a different name or delete these files and re-run this program.' % (name, name))

    publicKey, privateKey = generateKey(keySize)
    
    fo = open('%s_pubkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, publicKey[0], publicKey[1]))
    fo.close()

    fo = open('%s_privkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, privateKey[0], privateKey[1]))
    fo.close()

if __name__ == '__main__':
    main()

Generating p & q primes...
Generating e that is relatively prime to (p-1)*(q-1)...
Calculating d that is mod inverse of e...
Public key: (23750559398770470168350546458005623389958479972152237723291300075798918910853173186358246331732029899729669973085492377871699879779829566515206378642880970070796983872445867142566362536983983323088395318434674083842298781285705174183113883014598528036245125408753011211127718660060564726560810712297568615493604232784644151914254179177697516614336465960090043515982203248188660952128271464914530234033823758536569847012785879479301451814804164433140749630666560428013411966006562670492210165932336890301898351300271211229116996530103673901053684838350178193758579287422993931260429678571997686639144642104602933315661, 153268244329914282143126932557956421084318682481805748950538967921454060515212236671413413433545134380909796366130708261306017814762226427515795955786986521894267567199476317501275839633365496886975519192553566243322803789466272629855159527998

RSA implementation:

In [59]:
import sys, math


SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?.'

def getBlocksFromText(message, blockSize):
    # Converts a string message to a list of block integers.
    for character in message:
        if character not in SYMBOLS:
            print('ERROR: The symbol set does not have the character %s' % (character))
            sys.exit()
    blockInts = []
    for blockStart in range(0, len(message), blockSize):
        # Calculate the block integer for this block of text:
        blockInt = 0
        for i in range(blockStart, min(blockStart + blockSize, len(message))):
            blockInt += (SYMBOLS.index(message[i])) * (len(SYMBOLS) ** (i % blockSize))
        blockInts.append(blockInt)
    return blockInts


def getTextFromBlocks(blockInts, messageLength, blockSize):
    # Converts a list of block integers to the original message string.
    # The original message length is needed to properly convert the last
    # block integer.
    message = []
    for blockInt in blockInts:
        blockMessage = []
        for i in range(blockSize - 1, -1, -1):
            if len(message) + i < messageLength:
                # Decode the message string for the 128 (or whatever
                # blockSize is set to) characters from this block integer:
                charIndex = blockInt // (len(SYMBOLS) ** i)
                blockInt = blockInt % (len(SYMBOLS) ** i)
                blockMessage.insert(0, SYMBOLS[charIndex])
        message.extend(blockMessage)
    return ''.join(message)


def encryptMessage(message, key, blockSize):
    # Converts the message string into a list of block integers, and then
    # encrypts each block integer. Pass the PUBLIC key to encrypt.
    encryptedBlocks = []
    n, e = key

    for block in getBlocksFromText(message, blockSize):
        # ciphertext = plaintext ^ e mod n
        encryptedBlocks.append(pow(block, e, n))
    return encryptedBlocks


def decryptMessage(encryptedBlocks, messageLength, key, blockSize):
    # Decrypts a list of encrypted block ints into the original message
    # string. The original message length is required to properly decrypt
    # the last block. Be sure to pass the PRIVATE key to decrypt.
    decryptedBlocks = []
    n, d = key
    for block in encryptedBlocks:
        # plaintext = ciphertext ^ d mod n
        decryptedBlocks.append(pow(block, d, n))
    return getTextFromBlocks(decryptedBlocks, messageLength, blockSize)


def readKeyFile(keyFilename):
    # Given the filename of a file that contains a public or private key,
    # return the key as a (n,e) or (n,d) tuple value.
    fo = open(keyFilename)
    content = fo.read()
    fo.close()
    keySize, n, EorD = content.split(',')
    return (int(keySize), int(n), int(EorD))


def encryptAndWriteToFile(messageFilename, keyFilename, message, blockSize=None):
    # Using a key from a key file, encrypt the message and save it to a
    # file. Returns the encrypted message string.
    keySize, n, e = readKeyFile(keyFilename)
    if blockSize == None:
        # If blockSize isn't given, set it to the largest size allowed by the key size and symbol set size.
        blockSize = int(math.log(2 ** keySize, len(SYMBOLS)))
    # Check that key size is large enough for the block size:
    if not (math.log(2 ** keySize, len(SYMBOLS)) >= blockSize):
        sys.exit('ERROR: Block size is too large for the key and symbol set size. Did you specify the correct key file and encrypted file?')
    # Encrypt the message:
    encryptedBlocks = encryptMessage(message, (n, e), blockSize)

    # Convert the large int values to one string value:
    for i in range(len(encryptedBlocks)):
        encryptedBlocks[i] = str(encryptedBlocks[i])
    encryptedContent = ','.join(encryptedBlocks)

    # Write out the encrypted string to the output file:
    encryptedContent = '%s_%s_%s' % (len(message), blockSize, encryptedContent)
    fo = open(messageFilename, 'w')
    fo.write(encryptedContent)
    fo.close()
    # Also return the encrypted string:
    return encryptedContent


def readFromFileAndDecrypt(messageFilename, keyFilename):
    # Using a key from a key file, read an encrypted message from a file
    # and then decrypt it. Returns the decrypted message string.
    keySize, n, d = readKeyFile(keyFilename)


    # Read in the message length and the encrypted message from the file:
    fo = open(messageFilename)
    content = fo.read()
    messageLength, blockSize, encryptedMessage = content.split('_')
    messageLength = int(messageLength)
    blockSize = int(blockSize)

    # Check that key size is large enough for the block size:
    if not (math.log(2 ** keySize, len(SYMBOLS)) >= blockSize):
        sys.exit('ERROR: Block size is too large for the key and symbol set size. Did you specify the correct key file and encrypted file?')

    # Convert the encrypted message into large int values:
    encryptedBlocks = []
    for block in encryptedMessage.split(','):
        encryptedBlocks.append(int(block))

    # Decrypt the large int values:
    return decryptMessage(encryptedBlocks, messageLength, (n, d), blockSize)
    # If publicKeyCipher.py is run (instead of imported as a module) call
    # the main() function.

Test RSA:

In [60]:
import time
files = ['25k','50k','1mb','250mb','500mb']
modes = ['encrypt','decrypt']
for mo in modes:
    for file in files:
        print('file: {}'.format(file))
        filename = 'encryption_{}_rsa.txt'.format(file)
        start = time.time()
        if mo == 'encrypt':
            f = open(file.format(file), "r")
            message = f.read()
            pubKeyFilename = 'test_key_pubkey.txt'
            encryptedText = encryptAndWriteToFile(filename, pubKeyFilename, message)
  
        elif mo == 'decrypt':
            privKeyFilename = 'test_key_privkey.txt'
            decryptedText = readFromFileAndDecrypt(filename, privKeyFilename)
            d = open("decryption_{}_des".format(file), "a")
            d.write(decryptedText)
        end = time.time()
        print('time to {}: {}'.format(mo,end-start))

file: 25k
time to encrypt: 2.0241260528564453
file: 50k
time to encrypt: 4.008685827255249
file: 1mb
time to encrypt: 81.71896195411682
file: 250mb
time to encrypt: 20779.731500148773
file: 500mb
time to encrypt: 41032.963300943375
file: 25k
time to decrypt: 4.888412952423096
file: 50k
time to decrypt: 9.175631046295166
file: 1mb
time to decrypt: 185.42000484466553
file: 250mb
time to decrypt: 41614.34343290329
file: 500mb
time to decrypt: 81485.33118891716


The results:
1. Encryption:

| File Size |   25K  |   50K  |   1MB  |  250MB  |  500MB  |
|:---------:|:------:|:------:|:------:|:-------:|:-------:|
|   S-Des   | 0.0953 | 0.1781 | 3.8046 |  882.13 | 1734.09 |
|    Des    | 0.0050 | 0.0082 | 0.1503 | 43.4489 | 86.9020 |
|    RSA    | 2.0241 | 4.0086 | 81.718 | 20779.7 | 41032.9 |

2. Decryption:

| File Size |   25K  |   50K  |   1MB   |  250MB  |  500MB  |
|:---------:|:------:|:------:|:-------:|:-------:|:-------:|
|   S-Des   | 0.0854 | 0.1745 |  4.0060 | 874.153 | 1734.47 |
|    Des    | 0.0007 | 0.0008 |  0.0007 |  0.0008 |  0.0008 |
|    RSA    | 4.8884 | 9.1756 | 185.420 | 41614.3 | 81485.3 |

Note: All results in seconds.

Observations:

1. In S-Des, the encryption is faster than the decryption.
2. In Des, the decryption is faster than the encryption.
3. In RSA, the encryption is faster than the decryption by 4x.